# Adding a Library

To add a library in CMake, use the ```add_library()``` command and specify which source files should make up the library.

Rather than placing all of the source files in one directory, we can organize our project with one or more subdirectories. 

In this case, we will create a subdirectory specifically for our library. Here, we can add a new CMakeLists.txt file and one or more source files. 

In the top level CMakeLists.txt file, we will use the add_subdirectory() command to add the subdirectory to the build.

Once the library is created, it is connected to our executable target with target_include_directories() and target_link_libraries().



## Steps

1. In the CMakeLists.txt file in the MathFunctions directory, we create a library target called MathFunctions with add_library(). The source files for the library are passed as an argument to add_library(). This looks like the following line:

In [ ]:
# TODO 1: Add a library called MathFunctions with sources MathFunctions.cxx
# and mysqrt.cxx
# Hint: You will need the add_library command
add_library(MathFunctions MathFunctions.cxx mysqrt.cxx)

2. To make use of the new library we will add an add_subdirectory() call in the top-level CMakeLists.txt file so that the library will get built.

In [ ]:
# TODO 2: Use add_subdirectory() to add MathFunctions to this project
add_subdirectory(MathFunctions)

3. Next, the new library target is linked to the executable target using target_link_libraries().

In [ ]:
# TODO 3: Use target_link_libraries to link the library to our executable
target_link_libraries(Tutorial PUBLIC MathFunctions)

4. Finally we need to specify the library's header file location. Modify target_include_directories() to add the MathFunctions subdirectory as an include directory so that the MathFunctions.h header file can be found.

In [ ]:
# TODO 4: Add MathFunctions to Tutorial's target_include_directories()
# Hint: ${PROJECT_SOURCE_DIR} is a path to the project source. AKA This folder!

# add the binary tree to the search path for include files
# so that we will find TutorialConfig.h
target_include_directories(Tutorial PUBLIC
                          "${PROJECT_BINARY_DIR}"
                          "${PROJECT_SOURCE_DIR}/MathFunctions"
                          )

5. Now let's use our library. In tutorial.cxx, include MathFunctions.h:

In [ ]:
#include "MathFunctions.h"

6. Lastly, replace sqrt with the wrapper function mathfunctions::sqrt.

In [ ]:
 const double outputValue = mathfunctions::sqrt(inputValue);

## Adding and Option

Now let us add an option in the MathFunctions library to allow developers to select either the custom square root implementation or the built in standard implementation. 

While for the tutorial there really isn't any need to do so, for larger projects this is a common occurrence.

CMake can do this using the option() command. This gives users a variable which they can change when configuring their cmake build. 

This setting will be stored in the cache so that the user does not need to set the value each time they run CMake on a build directory.

- First create a variable USE_MYMATH using the option() command in MathFunctions/CMakeLists.txt. In that same file, use that option to pass a compile definition to the MathFunctions library.
- Then, update MathFunctions.cxx to redirect compilation based on USE_MYMATH.
- Lastly, prevent mysqrt.cxx from being compiled when USE_MYMATH is on by making it its own library inside of the USE_MYMATH block of MathFunctions/CMakeLists.txt.


## Steps

1. The first step is to add an option to MathFunctions/CMakeLists.txt. This option will be displayed in the cmake-gui and ccmake with a default value of ON that can be changed by the user.

In [ ]:
# TODO 7: Create a variable USE_MYMATH using option and set default to ON
option(USE_MYMATH "Use tutorial provided math implementation" ON)

2. Next, make building and linking our library with mysqrt function conditional using this new option.

Create an if() statement which checks the value of USE_MYMATH. Inside the if() block, put the target_compile_definitions() command with the compile definition USE_MYMATH.

In [ ]:
# TODO 8: If USE_MYMATH is ON, use target_compile_definitions to pass
# USE_MYMATH as a precompiled definition to our source files
if (USE_MYMATH)
  target_compile_definitions(MathFunctions PRIVATE "USE_MYMATH")
endif()

3. When USE_MYMATH is ON, the compile definition USE_MYMATH will be set. We can then use this compile definition to enable or disable sections of our source code.

The corresponding changes to the source code are fairly straightforward. In MathFunctions.cxx, we make USE_MYMATH control which square root function is used:

In [ ]:
// TODO 9: If USE_MYMATH is defined, use detail::mysqrt.
// Otherwise, use std::sqrt.
#ifdef USE_MYMATH
    return detail::mysqrt(x);
#else
    return std::sqrt(x);
#endif

4. Next, we need to include mysqrt.h if USE_MYMATH is defined.

In [ ]:
#ifdef USE_MYMATH
#  include "mysqrt.h"
#endif

5. Finally, we need to include cmath now that we are using std::sqrt.

In [ ]:
// TODO 11: include cmath
#include <cmath>

At this point, if USE_MYMATH is OFF, mysqrt.cxx would not be used but it will still be compiled because the MathFunctions target has mysqrt.cxx listed under sources.

There are a few ways to fix this. The first option is to use target_sources() to add mysqrt.cxx from within the USE_MYMATH block. Another option is to create an additional library within the USE_MYMATH block which is responsible for compiling mysqrt.cxx. For the sake of this tutorial, we are going to create an additional library.

6. First, from within USE_MYMATH create a library called SqrtLibrary that has sources mysqrt.cxx.

In [ ]:
# TODO 12: When USE_MYMATH is ON, add a library for SqrtLibrary with
# source mysqrt.cxx
add_library(SqrtLibrary STATIC
mysqrt.cxx
)

7. Next, we link SqrtLibrary onto MathFunctions when USE_MYMATH is enabled.

In [ ]:
# TODO 13: When USE_MYMATH is ON, link SqrtLibrary to the MathFunctions Library
target_link_libraries(MathFunctions PRIVATE SqrtLibrary)

8. Finally, we can remove mysqrt.cxx from our MathFunctions library source list because it will be pulled in when SqrtLibrary is included.